# Preprocesamiento

Se importan las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

Se carga el csv en un dataframe

In [ ]:
data = pd.read_csv('../bd/data2.csv', dtype={'rut': str})
data

Se elimina el rut, ya que no será utilizado para la regresión

In [ ]:
data_noRut = data.drop(['rut'], axis=1)
data_noRut

Se crean los codificadores de los datos categóricos

In [ ]:
le_rangoEdad = LabelEncoder()
le_sexo = LabelEncoder()
le_plan = LabelEncoder()
le_centro = LabelEncoder()
le_tipo = LabelEncoder()

Se codifican las variables

In [ ]:
rangoEdad_encoded = le_rangoEdad.fit_transform(data_noRut['rangoEdad'])
sexo_encoded = le_sexo.fit_transform(data_noRut['sexo'])
plan_encoded = le_plan.fit_transform(data_noRut['plan'])
centro_encoded = le_centro.fit_transform(data_noRut['centroFav'])
tipo_encoded = le_tipo.fit_transform(data_noRut['tipoFav'])

encoded_df = data_noRut[['visitas2018','visitas2019','visitas2020','visitas2021','visitas2022']]
encoded_df.insert(0, column='sexo_encoded', value=sexo_encoded)
encoded_df.insert(1, column='rangoEdad_encoded', value=rangoEdad_encoded)
encoded_df.insert(2, column='plan_encoded', value=plan_encoded)
encoded_df.insert(3, column='centro_encoded', value=centro_encoded)
encoded_df.insert(4, column='tipo_encoded', value=tipo_encoded)

encoded_df

Se separan los datos en un set de entrenamiento (80%) y un set de pruebas (20%)

In [ ]:
features = encoded_df.drop(['visitas2022'], axis=1)
labels = encoded_df[['visitas2022']]
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# Regresión Lineal

Se crea el modelo de regresión lineal

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)

Se crean las predicciones

In [ ]:
y_pred = reg.predict(X_test)
y_pred

Se muestran los indicadores del modelo

In [ ]:
MSE = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
R2 = r2_score(y_test,y_pred)
Score = reg.score(X_test,y_test)
print('MSE: ',MSE)
print('Score: ',Score)
print('R2_Score: ',R2)

# Postprocesamiento

Se redondea la predicción

In [ ]:
y_pred = np.round(y_pred).astype(int)

Se agrega la predicción a un dataframe para facilitar su visualización

In [ ]:
pred_data = pd.DataFrame(y_pred, columns=['pred_visitas2022'])
pred_data

Se unen los datos para entender de mejor manera los datos reales vs la predicción

In [ ]:
final_data = X_test
final_data = final_data.assign(real_visitas2022=y_test)
final_data.reset_index(drop=True,inplace=True)

inverse_sexo = le_sexo.inverse_transform(final_data['sexo_encoded'])
final_data.insert(0, column='sexo', value=inverse_sexo)
final_data.drop('sexo_encoded',axis=1,inplace=True)

inverse_rangoEdad = le_rangoEdad.inverse_transform(final_data['rangoEdad_encoded'])
final_data.insert(1, column='rangoEdad', value=inverse_rangoEdad)
final_data.drop('rangoEdad_encoded',axis=1,inplace=True)

inverse_plan = le_plan.inverse_transform(final_data['plan_encoded'])
final_data.insert(2, column='plan', value=inverse_plan)
final_data.drop('plan_encoded',axis=1,inplace=True)

inverse_centro = le_centro.inverse_transform(final_data['centro_encoded'])
final_data.insert(3, column='centro', value=inverse_centro)
final_data.drop('centro_encoded',axis=1,inplace=True)

inverse_tipo = le_tipo.inverse_transform(final_data['tipo_encoded'])
final_data.insert(4, column='tipo', value=inverse_tipo)
final_data.drop('tipo_encoded',axis=1,inplace=True)

final_df = pd.concat([final_data, pred_data],axis=1)
final_df.dropna(inplace=True)
final_df.reset_index(drop=True)
final_df[['visitas2018','visitas2019','visitas2020','visitas2021','real_visitas2022','pred_visitas2022']] = final_df[['visitas2018','visitas2019','visitas2020','visitas2021','real_visitas2022','pred_visitas2022']].astype(int)
final_df

Se muestra la suma y diferencia entre visitas reales del 2022 vs predecidas

In [ ]:
real = int(final_df['real_visitas2022'].sum())
pred = int(final_df['pred_visitas2022'].sum())
total = real + pred
diff = real - pred

aciertos = pd.DataFrame()
aciertos['aciertos'] = np.where((final_df['real_visitas2022'] == final_df['pred_visitas2022']),1,0)

print('Suma de las visitas reales del 2022: ', real)
print('Suma de las visitas predichas del 2022: ', pred)
print('Diferencia entre visitas reales vs predichas: ', diff)
print('Total de las visitas 2022: ', total)
#Nota: Que el total de visitas sea menor al total de filas de la base de datos está bien, ya que no está sumando los 0s.
print('-------------')
print('Aciertos de la predicción: ', aciertos['aciertos'].sum())
print('Fallos de la predicción: ', (aciertos['aciertos']==0).sum())
print('Porcentaje de aciertos: ', ((aciertos['aciertos'].sum())*100)/len(final_df))
print('Porcentaje de fallos: ', (((aciertos['aciertos']==0).sum())*100)/len(final_df))

Se crea una visualización de diferencia entre las visitas reales del 2022 vs las predecidas

In [ ]:
final_df['diff_visitas2022'] = final_df['real_visitas2022'] - final_df['pred_visitas2022']
dif = final_df[final_df.diff_visitas2022 >= 1]
dif.head(50)